In [18]:
import os
import sys
import math
import scipy
import importlib
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

import torch.nn.functional as F

%matplotlib
sys.path.insert(0, "/Users/andreaparra/Dropbox/4_Work/DataLamaCovid/projects/network_forcasting/GCN_COVID/GCN_pytorch")
print(sys.path)


Using matplotlib backend: MacOSX
['/Users/andreaparra/Dropbox/4_Work/DataLamaCovid/projects/network_forcasting/GCN_COVID/GCN_pytorch', '/Users/andreaparra/Dropbox/4_Work/DataLamaCovid/projects/network_forcasting/GCN_COVID/GCN_pytorch', '/Users/andreaparra/Dropbox/4_Work/DataLamaCovid/projects/network_forcasting', '/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.8/lib/python38.zip', '/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.8/lib/python3.8', '/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.8/lib/python3.8/lib-dynload', '', '/Users/andreaparra/Dropbox/4_Work/DataLamaCovid/projects/network_forcasting/venv/lib/python3.8/site-packages', '/Users/andreaparra/Dropbox/4_Work/DataLamaCovid/projects/network_forcasting/venv/lib/python3.8/site-packages/IPython/extensions', '/Users/andreaparra/.ipython']


In [29]:
base_path = "/Users/andreaparra/Dropbox/4_Work/DataLamaCovid/projects/network_forcasting/GCN_COVID/GCN_pytorch/out/"
data_set = "pred_window_1"

In [30]:
df_predictions = pd.read_csv(os.path.join(base_path, data_set, "predictions.csv"), sep="\t")
df_predictions.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,0.0,0.0000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.016298,0.0,0.0
1,0.0,0.0134,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.082462,0.0,0.072460,0.0,0.0
2,0.0,0.0000,0.0,0.0,0.0,0.0,0.254311,0.070231,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.113918,0.0,0.0
3,0.0,0.0000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.024314,0.0,0.000000,0.0,0.0
4,0.0,0.0000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0


In [27]:
df_targets = pd.read_csv(os.path.join(base_path, data_set, "target.csv"), sep="\t")
df_targets.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,0.191880,0.093266,0.591822,0.719201,0.173770,0.654531,0.134128,0.966768,0.426950,0.457147,0.499007,0.628024,0.319262,0.094035,0.773822,0.569242,0.560418,0.824094,0.463160
1,0.154538,0.236315,0.633965,0.204165,0.592868,0.174575,0.995530,0.920307,0.720406,0.358138,0.167373,0.279313,0.120385,0.943895,0.865590,0.488459,0.468403,0.337398,0.456450
2,0.189282,0.082011,0.685920,0.073806,0.687960,0.250358,0.823624,0.323471,0.255877,0.172863,0.507018,0.019115,0.518288,0.837075,0.900581,0.381410,0.462440,0.029662,0.083315
3,0.808965,0.106627,0.843631,0.074513,0.034171,0.841963,0.907591,0.454355,0.162568,0.466636,0.262548,0.095009,0.559711,0.572601,0.510655,0.041181,0.730027,0.665342,0.900803
4,0.044097,0.731094,0.805300,0.442610,0.303417,0.766107,0.731531,0.376013,0.718151,0.567012,0.787863,0.359071,0.659961,0.275629,0.806762,0.045465,0.011130,0.819438,0.489282


In [28]:
for node in df_predictions.columns:
    y_1 = df_predictions[node].values
    y_2 = df_targets[node].values

    error = []                          #MSE
    tot_error = 0                         
    for idx, y1 in enumerate(y_1):
        er = (y1 - y_2[idx]) ** 2
        error.append(math.sqrt(er))
        tot_error += er
    MSE = tot_error/len(error)

    x = df_predictions.index.values

    fig, (ax1, ax2) = plt.subplots(1,2, figsize=(15,8))

    # Plot predictions
    ax1.plot(x, y_1, color="indigo", label="prediction")
    ax1.plot(x, y_2, color="turquoise", label="target")
    ax1.set_title(f"Node number {node}")
    ax1.set_xlabel("Time (d)")
    ax1.set_ylabel("Number of cases (daily)")

    # Plot accuracy
    ax2.plot(x, error, color="tomato", label="error (squared error)")
    ax2.set_title(f"Error for node number {node}. (MSE: {round(MSE, 2)})")
    ax2.set_xlabel("Time (d)")
    ax2.set_ylabel("Error (squared error)")

    fig.legend()
    fig.savefig(os.path.join(base_path, data_set, f"{node}.jpeg"), bbox_inches='tight') 

In [10]:
errors = []
pred_dates = []
for window in range(1, 10):
    file_path = os.path.join(base_path, f"pred_window_{window}", "errors.csv")
    df = pd.read_csv(file_path, names=["error"])
    error = df["error"].mean()
    errors.append(error)
    pred_dates.append(window)

In [13]:
fig, ax = plt.subplots(1, 1)

ax.plot(pred_dates, errors, color="mediumseagreen")
ax.set_title(f"MSE")
ax.set_xlabel("Prediction window (d)")
ax.set_ylabel("Error (MSE)")

fig.savefig(os.path.join(base_path, "MSE.jpeg"), bbox_inches='tight') 
